In [2]:
import pickle
import numpy as np
import pandas as pd
!pip install scikit-multilearn
!pip install langdetect
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from langdetect import detect
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import GridSearchCV

In [3]:
def classify_genre(predictions):
    columns = ['Action','Adventure','Animation','Comedy','Crime','Documentary','Drama','Family','Fantasy','Mystery','Reality-TV','Romance','Sci-Fi','Thriller']
    ans = []
    for prediction in np.array(predictions):
        tmp = []
        for i in range(14):
            if prediction[i] == 1:
                tmp.append(columns[i])
        ans.append(tmp)
    return ans

In [4]:
def clean_text(corpuses):
    ans = []
    stop_words = stopwords.words('english')
    porter = PorterStemmer()
    for corpus in corpuses:
        corpus = ' '.join([porter.stem(x.lower()) for x in nltk.word_tokenize(corpus) if x.isalpha() and x not in stop_words])
        ans.append(corpus)
    return np.array(ans)

In [5]:
# Read a pickle file that contain tuples of only english movie plot and binary representation of genres
with open('only_en_x_y_data.pkl','rb') as fp: 
      x_data, y_data = pickle.load(fp)
X_train, X_test, Y_train, Y_test = train_test_split(x_data,y_data,test_size=0.33) # Split dataset with ratio 2:1

In [6]:
trans = TfidfVectorizer(min_df=2) # Create a TF-IDF Vector transformer
X_train_vec = trans.fit_transform(X_train) # Fit the transformer to train dataset and transform them
X_test_vec = trans.transform(X_test) # Transform test data set to vector based on train dataset

In [7]:
new_clf = MLkNN(k=1,s=0.5) # Create a MLkNN model with k = 1 and smoothness (alpha value) = 0.5
new_clf.fit(X_train_vec,lil_matrix(Y_train)) # Train the model (We have to use sparse matrix(lil_matrix))

MLkNN(ignore_first_neighbours=0, k=1, s=0.5)

In [8]:
prediction = new_clf.predict(X_test_vec) # Keep the result of prediction in a variable to avoid redundant calculation
accuracy_score(prediction,Y_test) # Use this one as a criteria to choose a classifier

0.6091585127201565

In [9]:
print(metrics.classification_report(prediction,Y_test)) # Use this data to analyze

              precision    recall  f1-score   support

           0       0.70      0.68      0.69      4211
           1       0.67      0.64      0.66      3509
           2       0.72      0.67      0.69      3970
           3       0.76      0.73      0.74     11363
           4       0.62      0.66      0.64      4635
           5       0.54      0.57      0.55      3683
           6       0.79      0.81      0.80     14830
           7       0.71      0.64      0.67      5704
           8       0.71      0.65      0.68      2555
           9       0.62      0.63      0.63      3875
          10       0.66      0.68      0.67      3892
          11       0.83      0.79      0.81      6594
          12       0.69      0.67      0.68      2972
          13       0.59      0.61      0.60      2846

   micro avg       0.71      0.70      0.71     74639
   macro avg       0.68      0.67      0.68     74639
weighted avg       0.71      0.70      0.71     74639
 samples avg       0.64   

//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [18]:
plot = input('') # Use input to be easier when we have to test
data = np.array([plot]) # numpy array is more friendly than list
data = trans.transform(clean_text(data))
classify_genre(new_clf.predict(data).todense())

ninja want to be hokage apart from father


[['Action', 'Adventure', 'Animation', 'Drama', 'Fantasy']]